## **Nugen Intelligence**
<img src="https://nugen.in/logo.png" alt="Nugen Logo" width="200"/>

Domain-aligned foundational models at industry leading speeds and zero-data retention! To learn more, visit [Nugen](https://docs.nugen.in/introduction)


## **NuGen LoRA Fine-Tuning Cookbook: A Beginner's Guide**

**Introduction**

Welcome to the NuGen LoRA Fine-Tuning Cookbook! This guide will walk you through the process of creating, training, deploying, and managing custom fine-tuned language models using NuGen's powerful Fine-Tuning API. With LoRA (Low-Rank Adaptation) fine-tuning, you can customize state-of-the-art language models to excel at your specific tasks without the computational expense of full model training.

**What is Fine-Tuning?**

Fine-tuning is the process of taking a pre-trained language model and customizing it for your specific use case. It's like teaching an already smart assistant your company's unique knowledge, terminology, and style.
What You'll Learn
In this cookbook, you'll learn how to:

**Prepare training data**
- Upload files to the NuGen platform
- Create datasets
- Train custom models
- Deploy and use your models

**What is LoRA Fine-Tuning?**

LoRA fine-tuning is a technique that allows you to adapt pre-trained language models to your specific needs by training only a small subset of parameters. This approach offers several advantages:

- Efficiency: Train models using significantly less computational resources
- Speed: Complete fine-tuning jobs in minutes or hours instead of days or weeks
- Cost-effectiveness: Reduce training and deployment costs dramatically
- Performance: Achieve comparable results to full fine-tuning for many applications

## **Getting Started**

Prerequisites

Before creating your first fine-tuned model, make sure you have:

1. A NuGen API key
2. Python 3.6+ installed
3. Required Python packages: requests, json, os, time, and python-dotenv

**Setting Up Your Environment**

First, create a .env file in your project directory to store your API key:

In [ ]:
NUGEN_API_KEY="nugen-************w"

## **The Fine-Tuning Workflow**

Fine-tuning a model with NuGen involves several key steps:

Prepare your training data

1. Upload your data file
2. Create a dataset from your file
3. Initialize a fine-tuning job
4. Monitor the training process
5. Deploy your fine-tuned model
6. Use your custom model

Let's walk through each step in detail.
Creating Your First Fine-Tuned Model

**Step 1: Setting Up Your Environment**

Let's start by setting up your development environment with the necessary dependencies

In [23]:
pip install --quiet requests python-dotenv

Note: you may need to restart the kernel to use updated packages.


**Step 2: Configuring Your Client**

Now, let's set up a simple client to interact with the NuGen API:

In [24]:
import requests
import json
import os
import time
from dotenv import load_dotenv

load_dotenv()

# Configuration settings
API_KEY = os.getenv("NUGEN_API_KEY")
BASE_URL = "https://api.nugen.in/finetune"
USERNAME = "your_email@example.com"  # Replace with your registered email

In [25]:
def api_call(method, endpoint, data=None, files=None):
    url = f"{BASE_URL}{endpoint}"
    headers = {"Authorization": f"Bearer {API_KEY}"}
    
    print(f"Making {method} request to {url}")
    
    if method == "GET":
        response = requests.get(url, headers=headers)
    elif method == "POST":
        if files:
            response = requests.post(url, headers=headers, files=files, data=data)
        else:
            headers["Content-Type"] = "application/json"
            response = requests.post(url, headers=headers, json=data)
    elif method == "DELETE":
        response = requests.delete(url, headers=headers)
    else:
        raise ValueError(f"Unsupported method: {method}")
    
    print(f"Status code: {response.status_code}")
    
    try:
        return response.json()
    except:
        return {"text": response.text}

For best results, we recommend:

- Including at least 50-100 examples for simple tasks
- Using diverse examples that cover the range of inputs your model will encounter
- Ensuring high-quality responses that model the exact output style you want

Preparing Your Training Data

**Step 3: Creating a Training Dataset**

For fine-tuning, you need a dataset in JSONL format. Let's create a simple dataset for teaching a model to provide capital city information:

In [ ]:
def create_sample_file():
    """
    Creates a sample training file in JSONL format.
    JSONL means each line is a separate JSON object - perfect for training data!
    
    Each training example has:
    - instruction: What you want to ask the model
    - response: How you want the model to answer
    """
    sample_data = [
        {"instruction": "What is the capital of France?", "response": "The capital of France is Paris."},
        {"instruction": "What is the capital of Italy?", "response": "The capital of Italy is Rome."},
        {"instruction": "What is the capital of Spain?", "response": "The capital of Spain is Madrid."}
    ]
    
    filename = "sample_finetune.jsonl"
    with open(filename, "w") as f:
        for item in sample_data:
            f.write(json.dumps(item) + "\n")
    
    print(f"✅ Created training file: {filename}")
    return filename


✅ Created training file: sample_finetune.jsonl


Why JSONL format? Each line represents one training example. It's like giving the AI flashcards - one question and answer per


Tip: For real-world applications, include at least 100 examples for best results. More examples lead to better model performance.

**File Management**

**Step 4: Uploading Your Dataset**

Let's upload our training file to the NuGen platform:

In [ ]:
def upload_file(username, filepath):
    """
    Uploads your training file to Nugen Intelligence.
    This is like putting your ingredients in Nugen's pantry.
    
    username: Your Nugen account username
    filepath: Path to your training file
    """
    print(f"\n=== Uploading {filepath} to Nugen Intelligence ===")
    
    with open(filepath, "rb") as f:
        upload_response = api_call(
            "POST", 
            f"/v1/files/{username}", 
            data={"purpose": "fine-tune"}, 
            files={"files": (filepath, f)}
        )
    
    print("Upload response:")
    print(json.dumps(upload_response, indent=2))
    
    # Extract the file ID for later use
    try:
        file_id = upload_response["uploaded_files"][0]["id"]
        print(f"✅ File uploaded successfully! File ID: {file_id}")
        return file_id
    except (KeyError, IndexError):
        print("❌ Failed to get file ID from response")
        return None



=== Uploading sample_finetune.jsonl to Nugen Intelligence ===
Making POST request to https://api.nugen.in/finetune/v1/files/your_email@example.com
Status code: 200
Upload response:
{
  "uploaded_files": [],
  "errors": null,
  "duplicate_files": {
    "sample_finetune.jsonl": "sample_finetune.jsonl"
  },
  "message": "The following files already exist: sample_finetune.jsonl"
}
❌ Failed to get file ID from response


**Step 5: Listing Your Files**

Let's check all the files you've uploaded to the platform:

In [ ]:
def list_files(username):
    """
    Shows all files you've uploaded to Nugen Intelligence.
    Like checking what's in your fridge before cooking!
    
    username: Your Nugen account username
    """
    print(f"\n=== Your Files in Nugen Intelligence ===")
    
    response = api_call("GET", f"/v1/files/{username}")
    print("Your uploaded files:")
    print(json.dumps(response, indent=2))
    


=== Your Files in Nugen Intelligence ===
Making GET request to https://api.nugen.in/finetune/v1/files/your_email@example.com
Status code: 200
Your uploaded files:
{
  "data": [
    {
      "id": "6829c49df323ec36221c9968",
      "object": "file",
      "bytes": 287,
      "created_at": 1747567774,
      "filename": "sample_finetune.jsonl",
      "purpose": "fine-tune",
      "sample_type": "pretrain",
      "num_lines": 3,
      "source": "upload",
      "username": "your_email@example.com",
      "file_location": "https://nugen-user-files--use1-az4--x-s3.s3.amazonaws.com/your_email@example.com/6829c49df323ec36221c9968_sample_finetune.jsonl",
      "file_hash": "2d543a77df04764cad59e0e5d751acd02ec745883289da813860bf88bc6b8960"
    }
  ],
  "object": "list"
}


**Dataset Creation**

**Step 6: Creating a Dataset from Your File**

Now that we've uploaded our file, let's create a dataset for fine-tunin

In [ ]:
def create_dataset(filepath):
    """
    Creates a dataset from your uploaded file.
    This prepares your training data for the fine-tuning process.
    
    filepath: Path to your training file
    """
    print(f"\n=== Creating Dataset from {filepath} ===")
    
    with open(filepath, "rb") as f:
        dataset_response = api_call(
            "POST", 
            "/v1/datasets/", 
            files={"file": (filepath, f)}
        )
    
    print("Dataset creation response:")
    print(json.dumps(dataset_response, indent=2))
    
    try:
        dataset_id = dataset_response["dataset_id"]
        print(f"✅ Dataset created successfully! Dataset ID: {dataset_id}")
        return dataset_id
    except KeyError:
        print("❌ Failed to get dataset ID from response")
        return None
    



=== Creating Dataset from sample_finetune.jsonl ===
Making POST request to https://api.nugen.in/finetune/v1/datasets/
Status code: 200
Dataset creation response:
{
  "dataset_id": "dataset-4632556509c3481e84642b90",
  "info": {
    "Name": "dataset-4632556509c3481e84642b90",
    "Status": "OK",
    "State": "READY",
    "Example Count": "3",
    "Create Time": "2025-05-19 10:24:00"
  },
  "status": "success",
  "raw_outputs": {
    "create": "",
    "info": "Name: accounts/nugen-intelligence/datasets/dataset-4632556509c3481e84642b90\nCreate Time: 2025-05-19 10:24:00\nState: READY\nStatus: OK\nExample Count: 3\nUser Uploaded:\nFormat: FORMAT_UNSPECIFIED\nCreated By: courteasyteam@gmail.com\nUpdate Time: 2025-05-19 10:24:01\n"
  }
}
✅ Dataset created successfully! Dataset ID: dataset-4632556509c3481e84642b90


**Step 7: Checking Dataset Status**

Let's verify the dataset is ready for training:

In [ ]:
def get_dataset_status(dataset_id):
    """
    Checks the status of your dataset.
    Nugen needs time to process and validate your training data.
    """
    print(f"\n=== Checking Dataset Status: {dataset_id} ===")
    
    dataset_status = api_call("GET", f"/v1/datasets/{dataset_id}")
    print("Dataset status:")
    print(json.dumps(dataset_status, indent=2))
    
    return dataset_status   



=== Checking Dataset Status: dataset-4632556509c3481e84642b90 ===
Making GET request to https://api.nugen.in/finetune/v1/datasets/dataset-4632556509c3481e84642b90
Status code: 200
Dataset status:
{
  "dataset_id": "dataset-4632556509c3481e84642b90",
  "info": {
    "Name": "dataset-4632556509c3481e84642b90",
    "Status": "OK",
    "State": "READY",
    "Example Count": "3",
    "Create Time": "2025-05-19 10:24:00"
  },
  "raw_output": "Name: accounts/nugen-intelligence/datasets/dataset-4632556509c3481e84642b90\nCreate Time: 2025-05-19 10:24:00\nState: READY\nStatus: OK\nExample Count: 3\nUser Uploaded:\nFormat: FORMAT_UNSPECIFIED\nCreated By: courteasyteam@gmail.com\nUpdate Time: 2025-05-19 10:24:01\n"
}


{'dataset_id': 'dataset-4632556509c3481e84642b90',
 'info': {'Name': 'dataset-4632556509c3481e84642b90',
  'Status': 'OK',
  'State': 'READY',
  'Example Count': '3',
  'Create Time': '2025-05-19 10:24:00'},
 'raw_output': 'Name: accounts/nugen-intelligence/datasets/dataset-4632556509c3481e84642b90\nCreate Time: 2025-05-19 10:24:00\nState: READY\nStatus: OK\nExample Count: 3\nUser Uploaded:\nFormat: FORMAT_UNSPECIFIED\nCreated By: courteasyteam@gmail.com\nUpdate Time: 2025-05-19 10:24:01\n'}

**Step 7: Starting Your Fine-tuning Job**

This is where the magic happens! Let's train your custom model:

In [ ]:
def create_finetune_job(dataset_id):
    """
    Starts training your custom AI model.
    This is like putting your recipe in the oven and waiting for it to bake!
    
    dataset_id: The ID of your prepared dataset
    """
    print(f"\n=== Starting Fine-tuning Job ===")
    
    job_data = {
        "dataset_id": dataset_id,
        "ft_type": "text_completion",  # Type of fine-tuning
        "epochs": 2,  # How many times to go through your training data
        "base_model": "accounts/fireworks/models/llama-v3p1-8b-instruct"  # The base model to customize
    }
    
    job_response = api_call("POST", "/v1/finetune/jobs", data=job_data)
    print("Fine-tuning job response:")
    print(json.dumps(job_response, indent=2))
    
    try:
        job_id = job_response["job_id"]
        model_id = job_response["model_id"]
        print(f"✅ Fine-tuning job started!")
        print(f"Job ID: {job_id}")
        print(f"Your custom model ID: {model_id}")
        return job_id, model_id
    except KeyError:
        print("❌ Failed to get job ID or model ID from response")
        return None, None
    


=== Starting Fine-tuning Job ===
Making POST request to https://api.nugen.in/finetune/v1/finetune/jobs
Status code: 200
Fine-tuning job response:
{
  "job_id": "job-8c2b967114a44e97bd990db7f1c5",
  "model_id": "model-8c2b967114a44e97bd990db7f1",
  "settings": {
    "job_id": "job-8c2b967114a44e97bd990db7f1c5",
    "model_id": "model-8c2b967114a44e97bd990db7f1",
    "dataset": "dataset-4632556509c3481e84642b90",
    "base_model": "llama-v3p1-8b-instruct",
    "epochs": 2,
    "text_completion": {
      "input_template": "### INSTRUCTION: {instruction} ### RESPONSE:",
      "output_template": "{response}"
    }
  },
  "status": "created",
  "raw_output": "Name: accounts/username/fineTuningJobs/job-8c2b967114a44e97bd990db7f1c5\nCreate Time: 2025-05-19 10:26:30\nState: CREATING\nDataset: accounts/username/datasets/dataset-4632556509c3481e84642b90\nStatus: OK\nCreated By: courteasyteam@gmail.com\nModel Id: model-8c2b967114a44e97bd990db7f1\nText Completion:\n  Input Template: ### INSTRUCTIO

What's happening here?

- epochs: How many times Nugen will read through your training data
- base_model: The foundation model that Nugen will customize for you
- ft_type: The type of training (text completion means generating text responses)

**Step 8: Monitoring Your Training Progress**

Let's check how your model training is going:

In [ ]:
def wait_for_job_completion(job_id, max_wait_time=600, check_interval=30):
    """Wait for job to complete or reach a stable state"""
    start_time = time.time()
    
    while time.time() - start_time < max_wait_time:
        job_status = api_call("GET", f"/v1/finetune/jobs/{job_id}")
        state = job_status.get("state", "UNKNOWN")
        
        print(f"Job state: {state}")
        
        if state in ["COMPLETED", "FAILED", "CANCELLED"]:
            return job_status
        elif state in ["RUNNING", "PENDING"]:
            print(f"Waiting {check_interval} seconds before next check...")
            time.sleep(check_interval)
        else:
            print(f"Unknown state: {state}")
            time.sleep(check_interval)
    
    print("Timeout waiting for job completion")
    return None

def get_job_status(job_id):
    """
    Checks the progress of your fine-tuning job.
    Like checking if your cake is done baking!
    """
    print(f"\n=== Checking Training Progress: {job_id} ===")
    
    job_status = api_call("GET", f"/v1/finetune/jobs/{job_id}")
    print("Training progress:")
    print(json.dumps(job_status, indent=2))
    
    return job_status



=== Checking Training Progress: job-8c2b967114a44e97bd990db7f1c5 ===
Making GET request to https://api.nugen.in/finetune/v1/finetune/jobs/job-8c2b967114a44e97bd990db7f1c5
Status code: 200
Training progress:
{
  "job_id": "job-8c2b967114a44e97bd990db7f1c5",
  "model_id": "model-8c2b967114a44e97bd990db7f1",
  "status": {
    "raw": "OK"
  },
  "state": "PENDING",
  "details": {
    "Name": "job-8c2b967114a44e97bd990db7f1c5",
    "Create Time": "2025-05-19 10:26:30",
    "State": "PENDING",
    "Dataset": "dataset-4632556509c3481e84642b90",
    "Model Id": "model-8c2b967114a44e97bd990db7f1",
    "Base Model": "accounts/username/models/llama-v3p1-8b-instruct",
    "Status": {
      "raw": "OK"
    },
    "Created By": null,
    "Text Completion": "",
    "Input Template": "### INSTRUCTION: {instruction} ### RESPONSE:",
    "Output Template": "{response}",
    "Epochs": "2",
    "Learning Rate": "0.0001",
    "Lora Rank": "8",
    "Batch Size": "16",
    "Evaluation Split": "0"
  }
}


{'job_id': 'job-8c2b967114a44e97bd990db7f1c5',
 'model_id': 'model-8c2b967114a44e97bd990db7f1',
 'status': {'raw': 'OK'},
 'state': 'PENDING',
 'details': {'Name': 'job-8c2b967114a44e97bd990db7f1c5',
  'Create Time': '2025-05-19 10:26:30',
  'State': 'PENDING',
  'Dataset': 'dataset-4632556509c3481e84642b90',
  'Model Id': 'model-8c2b967114a44e97bd990db7f1',
  'Base Model': 'accounts/username/models/llama-v3p1-8b-instruct',
  'Status': {'raw': 'OK'},
  'Created By': None,
  'Text Completion': '',
  'Input Template': '### INSTRUCTION: {instruction} ### RESPONSE:',
  'Output Template': '{response}',
  'Epochs': '2',
  'Learning Rate': '0.0001',
  'Lora Rank': '8',
  'Batch Size': '16',
  'Evaluation Split': '0'}}

**Best Practices for Fine-Tuning**

Effective Training Data Preparation

1. Quality over quantity: A smaller set of high-quality examples often outperforms a larger set of lower-quality examples.
2. Cover the input space: Include examples that represent the full range of inputs your model will encounter.
3. Consistent formatting: Keep formatting consistent between examples to help the model learn patterns effectively.
4. Balance the dataset: Ensure that different types of requests are proportionally represented.
5. Include edge cases: Add examples for rare or challenging scenarios to improve robustness.

**Hyperparameter Optimization**

While LoRA fine-tuning requires fewer hyperparameter adjustments than full fine-tuning, these settings can still impact performance:

1. Epochs: Start with 2-3 epochs for most applications. More epochs can help with complex tasks but risk overfitting.
2. Learning rate: The default learning rate works well for most cases, but you can experiment with different values for specific tasks.
3. Batch size: Larger batch sizes can improve training stability but require more memory. The default is usually appropriate.



**Step 9: Viewing All Your Training Jobs**

See all the fine-tuning jobs you've started:


In [ ]:
def list_finetune_jobs():
    """
    Shows all your fine-tuning jobs in Nugen Intelligence.
    Like a list of all the dishes you've ever cooked!
    """
    print(f"\n=== All Your Fine-tuning Jobs ===")
    
    jobs_list = api_call("GET", "/v1/fine-tuning-jobs")
    print("Your fine-tuning jobs:")
    print(json.dumps(jobs_list, indent=2))
    
    return jobs_list


=== All Your Fine-tuning Jobs ===
Making GET request to https://api.nugen.in/finetune/v1/fine-tuning-jobs
Status code: 200
Your fine-tuning jobs:
{
  "success": true,
  "jobs": [
    {
      "job_id": "job-8c2b967114a44e97bd990db7f1c5",
      "display_name": "",
      "create_date": "2025-05-19",
      "create_time": "10:26:30",
      "state": "PENDING"
    },
    {
      "job_id": "job-36a136adae104346a473bcdfd821",
      "display_name": "",
      "create_date": "2025-05-19",
      "create_time": "10:24:55",
      "state": "RUNNING"
    },
    {
      "job_id": "job-2d73b5986cf64985992ce7e67a52",
      "display_name": "",
      "create_date": "2025-05-18",
      "create_time": "11:30:05",
      "state": "COMPLETED"
    },
    {
      "job_id": "job-5759de5d05c142e9bb2d0a7fed37",
      "display_name": "",
      "create_date": "2025-05-18",
      "create_time": "08:24:05",
      "state": "COMPLETED"
    },
    {
      "job_id": "job-9d8f41e285e14a9085b5f8bd0fb2",
      "display_name": 

{'success': True,
 'jobs': [{'job_id': 'job-8c2b967114a44e97bd990db7f1c5',
   'display_name': '',
   'create_date': '2025-05-19',
   'create_time': '10:26:30',
   'state': 'PENDING'},
  {'job_id': 'job-36a136adae104346a473bcdfd821',
   'display_name': '',
   'create_date': '2025-05-19',
   'create_time': '10:24:55',
   'state': 'RUNNING'},
  {'job_id': 'job-2d73b5986cf64985992ce7e67a52',
   'display_name': '',
   'create_date': '2025-05-18',
   'create_time': '11:30:05',
   'state': 'COMPLETED'},
  {'job_id': 'job-5759de5d05c142e9bb2d0a7fed37',
   'display_name': '',
   'create_date': '2025-05-18',
   'create_time': '08:24:05',
   'state': 'COMPLETED'},
  {'job_id': 'job-9d8f41e285e14a9085b5f8bd0fb2',
   'display_name': '',
   'create_date': '2025-05-15',
   'create_time': '06:57:17',
   'state': 'COMPLETED'},
  {'job_id': 'job-13c66494a4c84e06adfc4aa7a5ed',
   'display_name': '',
   'create_date': '2025-05-14',
   'create_time': '18:28:58',
   'state': 'COMPLETED'},
  {'job_id': 'job-

**Step 10: Viewing All Your Datasets**

Check all the datasets you've created:


In [ ]:
def list_datasets():
    """
    Shows all your datasets in Nugen Intelligence.
    Your collection of training ingredients!
    """
    print(f"\n=== All Your Datasets ===")
    
    datasets_list = api_call("GET", "/v1/datasets")
    print("Your datasets:")
    print(json.dumps(datasets_list, indent=2))
    
    return datasets_list


=== All Your Datasets ===
Making GET request to https://api.nugen.in/finetune/v1/datasets
Status code: 200
Your datasets:
{
  "success": true,
  "datasets": [
    {
      "name": "dataset-0144e18c6a9f402d94851d28",
      "create_date": "2025-05-10",
      "create_time": "22:05:03",
      "state": "READY",
      "display_name": ""
    },
    {
      "name": "dataset-03bc75fe5d574918a018f4ec",
      "create_date": "2025-05-14",
      "create_time": "18:28:36",
      "state": "READY",
      "display_name": ""
    },
    {
      "name": "dataset-09864187ea164970b575d1c0",
      "create_date": "2025-05-10",
      "create_time": "21:54:42",
      "state": "READY",
      "display_name": ""
    },
    {
      "name": "dataset-0a8ebb32d674456d9da4be0d",
      "create_date": "2025-05-10",
      "create_time": "21:55:40",
      "state": "READY",
      "display_name": ""
    },
    {
      "name": "dataset-18c8c82eb6da415a87d8b65e",
      "create_date": "2025-05-10",
      "create_time": "21:59:2

{'success': True,
 'datasets': [{'name': 'dataset-0144e18c6a9f402d94851d28',
   'create_date': '2025-05-10',
   'create_time': '22:05:03',
   'state': 'READY',
   'display_name': ''},
  {'name': 'dataset-03bc75fe5d574918a018f4ec',
   'create_date': '2025-05-14',
   'create_time': '18:28:36',
   'state': 'READY',
   'display_name': ''},
  {'name': 'dataset-09864187ea164970b575d1c0',
   'create_date': '2025-05-10',
   'create_time': '21:54:42',
   'state': 'READY',
   'display_name': ''},
  {'name': 'dataset-0a8ebb32d674456d9da4be0d',
   'create_date': '2025-05-10',
   'create_time': '21:55:40',
   'state': 'READY',
   'display_name': ''},
  {'name': 'dataset-18c8c82eb6da415a87d8b65e',
   'create_date': '2025-05-10',
   'create_time': '21:59:25',
   'state': 'READY',
   'display_name': ''},
  {'name': 'dataset-21b693c2a0ba44e798091151',
   'create_date': '2025-05-18',
   'create_time': '08:47:11',
   'state': 'READY',
   'display_name': ''},
  {'name': 'dataset-26c4d7eb3a3f40d38d0160e4',

**Step 11:  Viewing All Your Models**

See all the custom models you've created:

In [ ]:
def get_model_status_safe(model_id):
    """
    Get model status with comprehensive error handling.
    This replaces the original get_model_status function.
    """
    print(f"\n=== Getting Model Status for {model_id} ===")
    model_status = api_call("GET", f"/v1/models/{model_id}")
    
    # Handle case where api_call returns a string instead of dict
    if isinstance(model_status, str):
        print("⚠️  Received string response instead of JSON:")
        print(model_status)
        return None
    
    # Check if it's a dict with error information
    if isinstance(model_status, dict):
        # Check for errors in nested structure
        status_info = model_status.get("info", {})
        if isinstance(status_info, dict):
            status_obj = status_info.get("Status", {})
            if isinstance(status_obj, dict) and status_obj.get("error"):
                print("⚠️  Model not ready yet or error occurred:")
                print(json.dumps(model_status, indent=2))
                return None
        
        print("✅ Model status retrieved successfully:")
        print(json.dumps(model_status, indent=2))
        return model_status
    else:
        print("⚠️  Unexpected response type:")
        print(f"Type: {type(model_status)}")
        print(f"Content: {model_status}")
        return None

def list_models():
    """
    Shows all your custom models in Nugen Intelligence.
    Your collection of trained AI chefs!
    """
    print(f"\n=== All Your Custom Models ===")
    
    models_list = api_call("GET", "/v1/models")
    print("Your custom models:")
    print(json.dumps(models_list, indent=2))
    
    return models_list



=== Getting Model Status for model-8c2b967114a44e97bd990db7f1 ===
Making GET request to https://api.nugen.in/finetune/v1/models/model-8c2b967114a44e97bd990db7f1
Status code: 200
⚠️  Model not ready yet or error occurred:
{
  "model_id": "model-8c2b967114a44e97bd990db7f1",
  "info": {
    "Name": "model-8c2b967114a44e97bd990db7f1",
    "State": "Unknown",
    "Create Time": null,
    "Fine Tuning Job": null,
    "Status": {
      "error": "Model not found"
    }
  },
  "raw_output": "2025/05/19 10:27:59 Failed to execute: rpc error: code = NotFound desc = resource not found\n"
}


**Step 12:  Checking Model Details**

Get detailed information about a specific model:


In [36]:
def get_model_status(model_id):
    """
    Gets detailed information about your custom model.
    Like reading the recipe card for a specific dish.
    """
    print(f"\n=== Model Details: {model_id} ===")
    
    model_status = api_call("GET", f"/v1/models/{model_id}")
    print("Model details:")
    print(json.dumps(model_status, indent=2))
    
    return model_status

**Model Deployment**

**Step 13: Deploying Your Model**

Now that your model is trained, let's deploy it for use:

In [37]:
def deploy_model(model_id):
    """
    Deploys your custom model so you can start using it.
    Like opening your restaurant to customers!
    """
    print(f"\n=== Deploying Model: {model_id} ===")
    
    deploy_response = api_call("POST", f"/v1/models/{model_id}/deploy")
    print("Deployment response:")
    print(json.dumps(deploy_response, indent=2))
    
    return deploy_response

**Step 14: Listing Deployed Models**

Let's check which models are currently deployed:

In [38]:
def list_deployed_models():
    """
    Shows all your currently deployed models.
    Your active AI assistants ready to help!
    """
    print(f"\n=== Your Active Models ===")
    
    deployed_models = api_call("GET", "/v1/deployed-models")
    print("Your deployed models:")
    print(json.dumps(deployed_models, indent=2))
    
    return deployed_models

**Model Download and Management**

**Step 15: Downloading Model Weights**

If you need to use your model offline, you can download its weights:

In [39]:
def download_model_weights(model_id):
    """
    Downloads the weights (trained parameters) of your model.
    Like getting the secret recipe of your perfected dish!
    """
    print(f"\n=== Downloading Model Weights: {model_id} ===")
    
    weights_response = api_call("GET", f"/v1/models-weights/{model_id}")
    print("Weights download response:")
    print(json.dumps(weights_response, indent=2))
    
    return weights_response

**Step 16: Undeploying a Model**

When you're done using a model, you can undeploy it to free up resources:

In [40]:
def undeploy_model(model_id):
    """
    Stops your deployed model from serving requests.
    Like temporarily closing your restaurant.
    """
    print(f"\n=== Undeploying Model: {model_id} ===")
    
    undeploy_response = api_call("POST", f"/v1/models/{model_id}/undeploy")
    print("Undeploy response:")
    print(json.dumps(undeploy_response, indent=2))
    
    return undeploy_response

**Resource Cleanup**

**Step 17: Deleting Resources**

When you no longer need certain resources, you can delete them:

In [41]:
def delete_model(model_id):
    """
    Permanently deletes a model from Nugen Intelligence.
    ⚠️ Warning: This cannot be undone!
    """
    print(f"\n=== Deleting Model: {model_id} ===")
    
    delete_response = api_call("DELETE", f"/v1/models/{model_id}")
    print("Delete response:")
    print(json.dumps(delete_response, indent=2))
    
    return delete_response

def delete_dataset(dataset_id):
    """
    Permanently deletes a dataset from Nugen Intelligence.
    ⚠️ Warning: This cannot be undone!
    """
    print(f"\n=== Deleting Dataset: {dataset_id} ===")
    
    delete_response = api_call("DELETE", f"/v1/datasets/{dataset_id}")
    print("Delete response:")
    print(json.dumps(delete_response, indent=2))
    
    return delete_response

def delete_file(username, file_id):
    """
    Permanently deletes an uploaded file from Nugen Intelligence.
    ⚠️ Warning: This cannot be undone!
    """
    print(f"\n=== Deleting File: {file_id} ===")
    
    delete_response = api_call("DELETE", f"/v1/files/{username}/{file_id}")
    print("Delete response:")
    print(json.dumps(delete_response, indent=2))
    
    return delete_response

Putting It All Together: Complete Workflow

Now let's combine all these steps into a complete fine-tuning workflow:

In [58]:
def complete_finetuning_workflow():
    """
    The complete workflow for fine-tuning a model with Nugen Intelligence.
    Updated with proper timing to prevent 'Model not found' errors.
    """
    print("🔥 Starting Complete Fine-tuning Workflow with Nugen Intelligence 🔥")
    
    # Initialize variables to track created resources
    file_id = None
    dataset_id = None
    job_id = None
    model_id = None
    
    try:
        # Step 1: Prepare your training data
        sample_file = create_sample_file()
        
        # Step 2: Upload your file to Nugen
        file_id = upload_file(USERNAME, sample_file)
        
        # Step 3: Check your uploaded files
        list_files(USERNAME)
        
        # Step 4: Create a dataset from your file
        dataset_id = create_dataset(sample_file)
        
        if not dataset_id:
            print("❌ Failed to create dataset. Stopping workflow.")
            return None, None, None, None
        
        # Wait for Nugen to process your dataset
        print("⏳ Waiting for Nugen to process your dataset...")
        time.sleep(5)
        
        # Step 5: Check dataset status
        get_dataset_status(dataset_id)
        
        # Step 6: Start fine-tuning your model
        job_id, model_id = create_finetune_job(dataset_id)
        
        if not job_id or not model_id:
            print("❌ Failed to create fine-tuning job. Stopping workflow.")
            return file_id, dataset_id, None, None
        
        # Step 7: Wait for job completion (THIS IS THE KEY FIX!)
        print("\n🚀 Starting training and waiting for completion...")
        final_job_status = wait_for_job_completion(job_id, max_wait_time=1800)  # 30 minutes max
        
        if final_job_status:
            final_state = final_job_status.get("state")
            print(f"\n✅ Final job state: {final_state}")
            
            if final_state == "COMPLETED":
                print("🎉 Training completed successfully!")
                
                # Step 8: Now safely get model status
                print("\n=== Getting Final Job Details ===")
                final_job_details = get_job_status(job_id)
                
                model_status = get_model_status_safe(model_id)
                
                if model_status:
                    print("🎯 Model is ready for deployment and use!")
                    
                    # Optionally deploy the model
                    # deploy_response = deploy_model(model_id)
                    
                else:
                    print("⚠️ Model is not ready yet, please try again later")
                    
            elif final_state == "FAILED":
                print("❌ Training job failed!")
                print("Error details:")
                print(json.dumps(final_job_status.get("details", {}), indent=2))
                
            elif final_state == "CANCELLED":
                print("🛑 Training job was cancelled")
                
        else:
            print("⏰ Training did not complete within the timeout period")
            print("The job may still be running. Check back later with:")
            print(f"get_job_status('{job_id}')")
        
        # Step 9: Review everything you've created
        print("\n🔍 Let's review what you've created:")
        list_finetune_jobs()
        list_datasets()
        list_models()
        list_deployed_models()
        
    except Exception as e:
        print(f"❌ An error occurred during the workflow: {str(e)}")
        import traceback
        traceback.print_exc()
    
    finally:
        # Clean up the sample file from your computer
        if 'sample_file' in locals():
            try:
                os.remove(sample_file)
                print(f"✅ Cleaned up local file: {sample_file}")
            except OSError:
                print(f"Could not remove local file: {sample_file}")
    
    print("\n🎉 Congratulations! You've successfully completed the fine-tuning workflow!")
    print("Your custom AI model is ready to be deployed and used for your specific needs.")
    
    # Print summary
    print(f"\n📋 Summary of Created Resources:")
    print(f"File ID: {file_id}")
    print(f"Dataset ID: {dataset_id}") 
    print(f"Job ID: {job_id}")
    print(f"Model ID: {model_id}")
    
    # Return the IDs for further use
    return file_id, dataset_id, job_id, model_id


complete_finetuning_workflow()

🔥 Starting Complete Fine-tuning Workflow with Nugen Intelligence 🔥
✅ Created training file: sample_finetune.jsonl

=== Uploading sample_finetune.jsonl to Nugen Intelligence ===
Making POST request to https://api.nugen.in/finetune/v1/files/your_email@example.com
Status code: 200
Upload response:
{
  "uploaded_files": [],
  "errors": null,
  "duplicate_files": {
    "sample_finetune.jsonl": "sample_finetune.jsonl"
  },
  "message": "The following files already exist: sample_finetune.jsonl"
}
❌ Failed to get file ID from response

=== Your Files in Nugen Intelligence ===
Making GET request to https://api.nugen.in/finetune/v1/files/your_email@example.com
Status code: 200
Your uploaded files:
{
  "data": [
    {
      "id": "6829c49df323ec36221c9968",
      "object": "file",
      "bytes": 287,
      "created_at": 1747567774,
      "filename": "sample_finetune.jsonl",
      "purpose": "fine-tune",
      "sample_type": "pretrain",
      "num_lines": 3,
      "source": "upload",
      "userna

(None,
 'dataset-3495133f8ae8497fa38064a6',
 'job-453a2f3dca2d400195d4d4d0008d',
 'model-453a2f3dca2d400195d4d4d000')

**Best Practices for Fine-Tuning**

1. Quality Data: Your model is only as good as the data you train it on. Focus on high-quality, diverse examples.
2. Epochs: Start with 2-3 epochs. Too many epochs can lead to overfitting, while too few may result in underfitting.
3. Testing: Always test your model thoroughly before deploying it to production.
4. Resource Management: Delete unused models and datasets to save resources and keep your workspace clean.
5. Security: Keep your API keys secure and never expose them in client-side code.

**Conclusion**

Congratulations! You've learned how to create, train, and deploy custom AI models using NuGen's fine-tuning API. This powerful capability allows you to build AI solutions tailored to your specific needs, whether you're creating a customer service bot, a content generator, or a specialized knowledge assistant.
Remember that fine-tuning is an iterative process. Don't be afraid to experiment with different datasets, parameters, and approaches to achieve the best results for your use case.
Happy fine-tuning with NuGen AI!